<h1><center>Laboratorio N° 03: Pivoting y Grouping Sets

<h2>Parte 01. Escribir consultas que utilizan PIVOT.

1.1. Abrir la base de datos TSQL.

In [ ]:
USE TSQL;
GO

1.2. Crear un vista de categorías de productos, sus cantidaddes por año.

1.3. Probar la vista creada.

1.4. Utilizar PIVOT entre categorías y años de ordenes.

1.5. Para utilizar UNPIVOT, crear una tabla con el resultado de la consulta con PIVOT anterior.

1.6. Probrar la tabla generada

1.7. Utilizar UNPIVOT.

1.8. Limpiar los cambios realizados.


<h2>Parte 02. Escribir consultas que utilizan UNPIVOT.

2.1. Ejecutar la siguiente vistas

2.2. Elaborar una consulta sin utilizar GROUPING SETS.

2.3. Consultar con GROUPING SETS.

2.4. Consultar con CUBE.

2.5. Consultar con ROLLUP

2.6. Utilizar Grouping_ID.

2.7. Finalmente limpiar los cambios

<h2>Ejercicio 03: Escribir consultas que usan las clausulas GROUPING SETS, CUBE y ROLLUP

3.1. Escribir una sentencia SELECT contra la tabla Sales.Customers y traer las columnas country, city
y una columna calculada noofcustomers como una cantidad de clientes. Traer multiples grouping
sets basados en las columnas country y city, la columna country, la columna city, y una columna con un grouping set vacio.

3.2. Escribir una sentencia SELECT contra la vista Sales.OrderValues y traer las columnas:
<li>Año de la columna orderdate como orderyear.
<li>Mes de la columna orderdate como ordermonth.
<li>Día de la columna orderdate como orderday.
<li>Valor de Total de Ventas utilizando la columna val como salesvalue.
<li>Devolver todas los posibles  grouping sets basados en las columnas orderyear, ordermonth, y orderday.

3.3. Copiar la consulta previa y modificarla para utilizar la clausula ROLLUP en vez de CUBE.

3.4. Escribir una sentencia SELECT contra la vista Sales.OrderValues y traer las columnas:
<li>Columna Calculada con el alias groupid (usar la función GROUPING_ID con los campos order year y order month como parametros).
<li>Año de la columna orderdate como orderyear.
<li>Mes de la columna orderdate como ordermonth.
<li>Valor Total de Ventas utilizando la columna val como salesvalue.
<li>Desde Año y mes month forman una jerarquia, retornar todos los grouping sets interesantes basados en las columnas orderyear y ordermonth y ordenar el resultado por groupid, orderyear, and ordermonth